In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Input,Dense,Flatten,Lambda,Conv2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob


In [4]:
print(tf.__version__)

2.1.0


In [5]:
image_size=[224,224]

train_path=r"C:\Users\Lenovo\Downloads\flowers\train"
test_path=r"C:\Users\Lenovo\Downloads\flowers\test"

In [6]:
vgg19=VGG19(input_shape=image_size+[3],weights='imagenet',include_top=False)

In [7]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
#dont train the existing weight so, we will use

for layer in vgg19.layers:
    layer.trainable=False

In [9]:
#now flatten the output of vgg19

x=Flatten()(vgg19.output)

In [10]:
print(x)
print(vgg19.input)

Tensor("flatten/Identity:0", shape=(None, 25088), dtype=float32)
Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32)


In [11]:
prediction=Dense(5,activation='sigmoid')(x)
model=Model(inputs=vgg19.input, outputs=prediction)

In [12]:
print(prediction)

Tensor("dense/Identity:0", shape=(None, 5), dtype=float32)


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
from tensorflow.keras.layers import MaxPooling2D

In [15]:
#model=Sequential()
#model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation='relu',inpu))

In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
#use ImageDataGenerator to import images from folder

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=.2,
                                zoom_range=.2,
                                horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [19]:
training_set=train_datagen.flow_from_directory(r"C:\Users\Lenovo\Downloads\flowers\train",
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 4323 images belonging to 5 classes.


In [20]:
testing_set=test_datagen.flow_from_directory(r"C:\Users\Lenovo\Downloads\flowers\test",
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode='categorical')

Found 308 images belonging to 5 classes.


# fit model

In [21]:
history=model.fit(training_set,
                  validation_data=testing_set,
                  epochs=2,
                  steps_per_epoch=len(training_set),
                  validation_steps=len(testing_set))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 136 steps, validate for 10 steps
Epoch 1/2
136/136 [==============================] - 1559s 11s/step - loss: 0.3172 - accuracy: 0.8746 - val_loss: 0.2441 - val_accuracy: 0.8974
Epoch 2/2
136/136 [==============================] - 1538s 11s/step - loss: 0.2098 - accuracy: 0.9170 - val_loss: 0.2369 - val_accuracy: 0.8968


In [22]:
from tensorflow.keras.models import load_model

model.save('modelflower_vgg19.h5')

In [24]:
y_pred=model.predict(testing_set)

In [25]:
print(y_pred)

[[1.1912423e-02 1.5946269e-02 1.2035424e-02 1.1868036e-01 7.5510454e-01]
 [3.4786247e-02 2.4203553e-03 1.9399658e-02 3.3681057e-03 9.5752209e-01]
 [2.1628198e-01 7.5005412e-01 7.8712385e-03 1.8020090e-02 3.1723510e-02]
 ...
 [8.6438674e-03 3.6131181e-03 1.6550999e-03 8.7162852e-01 4.3121353e-02]
 [9.5342612e-01 4.9942896e-02 6.3666404e-05 9.4204629e-03 7.3868227e-03]
 [1.1250884e-01 1.1023963e-02 2.8071523e-02 1.4417297e-02 7.4720424e-01]]


In [26]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)
y_pred

array([4, 4, 1, 1, 2, 4, 3, 4, 0, 4, 4, 4, 4, 1, 0, 2, 3, 2, 1, 1, 4, 0,
       4, 2, 4, 2, 0, 1, 1, 2, 2, 4, 4, 4, 4, 0, 2, 0, 1, 4, 4, 4, 4, 0,
       4, 3, 0, 2, 1, 1, 0, 4, 1, 2, 4, 0, 0, 0, 0, 4, 1, 0, 4, 2, 0, 0,
       4, 3, 1, 0, 2, 4, 4, 0, 0, 4, 0, 0, 1, 0, 2, 4, 1, 0, 3, 0, 2, 1,
       3, 0, 4, 4, 1, 3, 4, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 3, 0, 2, 0,
       4, 0, 4, 4, 0, 0, 3, 0, 0, 4, 4, 0, 0, 2, 1, 1, 4, 4, 2, 2, 0, 0,
       1, 1, 0, 4, 3, 3, 0, 4, 4, 4, 0, 1, 3, 3, 0, 2, 1, 4, 4, 0, 1, 2,
       0, 0, 4, 3, 0, 4, 0, 4, 4, 2, 1, 0, 0, 2, 4, 0, 0, 4, 0, 2, 0, 0,
       4, 0, 3, 4, 0, 4, 1, 0, 3, 0, 4, 2, 1, 0, 2, 4, 4, 0, 1, 0, 1, 4,
       0, 4, 0, 4, 3, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 1, 2, 1, 3, 4, 3, 0,
       4, 3, 0, 1, 1, 0, 0, 1, 2, 4, 0, 0, 0, 0, 2, 3, 0, 4, 1, 1, 3, 0,
       0, 0, 3, 3, 4, 3, 1, 4, 0, 4, 1, 4, 0, 1, 0, 4, 0, 4, 0, 4, 1, 4,
       3, 4, 2, 0, 4, 0, 0, 2, 0, 0, 1, 4, 4, 4, 4, 4, 4, 3, 0, 0, 4, 1,
       2, 1, 4, 2, 3, 1, 4, 3, 0, 4, 2, 4, 1, 1, 3,

# check the prediction

In [27]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [28]:
model=load_model('modelflower_vgg19.h5')

In [49]:
img=image.load_img(r"C:\Users\Lenovo\Downloads\sunflower.jfif",target_size=(224,224))

In [50]:
x=image.img_to_array(img)

In [51]:
x.shape

(224, 224, 3)

In [52]:
x=x/255

# expand the image dimension

In [53]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [54]:
model.predict(img_data)

array([[2.0130160e-03, 9.9296391e-01, 1.8355577e-06, 3.6530799e-01,
        1.1692367e-05]], dtype=float32)

In [57]:
prediction=np.argmax(model.predict(img_data),axis=1)
prediction

array([1], dtype=int64)

In [58]:
if (prediction==0):
    print("Daisy")
elif (prediction==1):
    print('Dandelion')
elif (prediction==2):
    print('rose')
elif(prediction==3):
    print('sunflower')
else:
    print('tulip')

Dandelion
